In [2]:
import datetime

from google.cloud import bigquery
import pandas as pd
import pytz
from google.oauth2 import service_account
import numpy as np

In [3]:
credentials = service_account.Credentials.from_service_account_file('pax-11-366616-3e921bf3ac9e.json')

In [4]:
client = bigquery.Client(credentials=credentials)

In [5]:
!pip3 install pymysql

In [6]:
import pymysql

In [7]:

# Connect to the database
connection = pymysql.connect(host='34.148.88.4',
                         user='root',
                         password='tejaswini',
                         db='oltp_schema')

# create cursor
cursor=connection.cursor()

In [8]:
df_order_details=pd.read_sql(f'''select * from order_details''',connection)
df_consumer_master=pd.read_sql(f'''select * from customer_master''',connection)
df_order_items=pd.read_sql(f'''select * from order_items''',connection)
df_product_master=pd.read_sql(f'''select * from product_master''',connection)

C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\dhara\PYTHON\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 c

In [9]:
def insert_dim_order(df_order_details):
    g = df_order_details
    #g = df_order_details.groupby("orderid").tail(1)
    x = g[["orderid","order_status_update_timestamp","order_status"]]
    dim_order = x.reset_index()
    del dim_order["index"]
    return dim_order

In [10]:
def insert_dim_customer(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_customer =df_consumer_master[["customerid","name","address_id"]]
    dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
    dim_customer["end_date"] = np.nan
    dim_customer["end_date"]=pd.to_datetime(dim_customer["end_date"])
    return dim_customer

In [11]:
def insert_dim_address(df_consumer_master):
    df_consumer_master["address_id"] = list(range(1,1001))
    dim_address = df_consumer_master[["address_id","address","city","state","pincode"]]
    return dim_address

In [12]:
def insert_dim_product(df_product_master):
    dim_product = df_product_master
    dim_product["start_date"]=np.nan
    dim_product["start_date"]=pd.to_datetime(dim_product["start_date"])
    dim_product["end_date"]=np.nan
    dim_product["end_date"]=pd.to_datetime(dim_product["end_date"])
    return dim_product

In [13]:
def insert_f_order_details(df_order_details,df_order_items):
    f_order_details = pd.merge(df_order_details.groupby("orderid").tail(1), df_order_items, how='inner')[["orderid","order_status_update_timestamp","productid","quantity"]]
    f_order_details.columns = ["orderid","order_delivery_timestamp","productid","quantity"]
    return f_order_details

In [43]:
def insert_fact_daily_orders(delta_orders,delta_customer,delta_products,fact_order_details):
    fields = ['customerid','orderid','order_received_timestamp','order_delivery_timestamp','pincode','order_amount','item_count','order_delivery_time_seconds']
    fact_daily_orders = pd.DataFrame(columns=fields)
    
    received_df = delta_orders.where(delta_orders['order_status']=='Received').dropna()[['customerid','orderid','order_status_update_timestamp']]
    delivered_df = delta_orders.where(delta_orders['order_status']=='Delivered').dropna()[['customerid','orderid','order_status_update_timestamp']]
    temp_df = pd.merge(received_df, delivered_df, on=['customerid','orderid'], how='left')
    
    temp_df['order_delivery_time_seconds']=pd.to_datetime(temp_df['order_status_update_timestamp_y'])-pd.to_datetime(temp_df['order_status_update_timestamp_x'])
    temp_df['order_delivery_time_seconds']=temp_df['order_delivery_time_seconds']/np.timedelta64(1,'s')
    
    temp_df.rename(columns = {'order_status_update_timestamp_x':'order_received_timestamp',
                        'order_status_update_timestamp_y':'order_delivery_timestamp'}, inplace = True)
    
    temp_df = pd.merge(temp_df, delta_customer, on=['customerid'], how='left')
    temp_df = temp_df.drop(['name','address','city','state','update_timestamp'],axis=1)
    
    #df= pd.merge(temp_df,delta_products, on ='productid',how='left')
    
    #Derive Order_amount column
    df= pd.merge(fact_order_details,delta_products, on ='productid',how='left')
    df['order_amount'] = df['quantity']*df['rate']
    df = df.drop(['productcode','productname','sku','isactive'],axis=1)
    
    amounts= df[['orderid','order_amount']]
    amounts= amounts.groupby('orderid').sum()
    
    quants = df[['orderid','quantity']]
    quants= quants.groupby('orderid').sum()
    
    temp_df = pd.merge(temp_df,amounts,on='orderid',how='left')
    fact_daily_orders = pd.merge(temp_df,quants,on='orderid',how='left')
    fact_daily_orders.rename(columns = {'quantity':'item_count'}, inplace = True)
    fact_daily_orders.fillna(0, inplace=True)
    fact_daily_orders.drop(['address_id'],axis=1, inplace=True)
    
    return fact_daily_orders

In [44]:
def FactOrderDetailsTransform(delta_orders,delta_order_items):
    delivered_df = delta_orders.where(delta_orders['order_status']=='Delivered').dropna()[['customerid','orderid','order_status_update_timestamp']]
    df = pd.merge(delta_order_items,delivered_df,on='orderid',how='left')
    df = df.drop(['customerid'],axis=1)
    df.rename(columns = {'order_status_update_timestamp':'order_delivery_timestamp'}, inplace = True)
    df.index=np.arange(1,len(df)+1) 
    
    return df

In [45]:
fact_order_details = FactOrderDetailsTransform(df_order_details, df_order_items)

In [46]:
fact_daily_orders = insert_fact_daily_orders(df_order_details,df_consumer_master,df_product_master,fact_order_details)

In [47]:
fact_daily_orders.head()

,customerid,orderid,order_received_timestamp,order_delivery_timestamp,order_delivery_time_seconds,pincode,order_amount,item_count
0,50.0,1.0,2022-11-19 16:48:54,2022-11-20 14:22:07,77593.0,111463,5968,5
1,990.0,2.0,2022-03-30 02:00:36,2022-03-30 18:07:56,58040.0,843590,10930,9
2,627.0,3.0,2022-07-22 06:23:57,2022-07-23 17:28:13,126256.0,640651,9365,6
3,916.0,4.0,2022-09-07 11:36:35,2022-09-07 16:20:29,17034.0,363894,6892,7
4,24.0,5.0,2022-03-10 00:00:55,2022-03-10 20:49:25,74910.0,775124,23616,15


In [15]:
dim_order = insert_dim_order(df_order_details)

In [16]:
df_order_items.head()

,orderid,productid,quantity
0,1,25,1
1,1,26,4
2,2,32,4
3,2,33,5
4,3,22,1


In [17]:
df_order_items.iloc[0,2]

1

In [18]:
dim_customer = insert_dim_customer(df_consumer_master)

C:\Users\dhara\AppData\Local\Temp\ipykernel_12284\43446549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["start_date"]= (df_consumer_master['update_timestamp'].dt.date).astype('datetime64')
C:\Users\dhara\AppData\Local\Temp\ipykernel_12284\43446549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_customer["end_date"] = np.nan
C:\Users\dhara\AppData\Local\Temp\ipykernel_12284\43446549.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [19]:
dim_address = insert_dim_address(df_consumer_master)

In [20]:
dim_product=insert_dim_product(df_product_master)

In [21]:
f_order_details = insert_f_order_details(df_order_details,df_order_items)


In [59]:
tableRef1 = client.dataset("star_schema").table("dim_order")
client.load_table_from_dataframe(dim_order,tableRef1)

LoadJob<project=pax-11-366616, location=US, id=7f85af4b-4d7d-4fd0-8bcf-6c56b5af5ba2>

In [60]:
tableRef2 = client.dataset("star_schema").table("dim_customer")
client.load_table_from_dataframe(dim_customer,tableRef2)

LoadJob<project=pax-11-366616, location=US, id=74f61190-b72d-4232-9496-b5634debb38e>

In [61]:
tableRef3 = client.dataset("star_schema").table("dim_address")
client.load_table_from_dataframe(dim_address,tableRef3)


LoadJob<project=pax-11-366616, location=US, id=386c80aa-6851-411c-bf53-e30892915bcc>

In [62]:
tableRef4 = client.dataset("star_schema").table("dim_product")
client.load_table_from_dataframe(dim_product,tableRef4)

LoadJob<project=pax-11-366616, location=US, id=570d24f1-0aad-4ae7-9742-d8201d3ebaee>

In [63]:
tableRef5 = client.dataset("star_schema").table("f_order_details")
client.load_table_from_dataframe(f_order_details,tableRef5)

LoadJob<project=pax-11-366616, location=US, id=21b05df4-779d-45e7-917f-e9edf5769b65>

In [48]:
tableRef6 = client.dataset("star_schema").table("fact_daily_orders")
client.load_table_from_dataframe(fact_daily_orders,tableRef6)

LoadJob<project=pax-11-366616, location=US, id=d83f3f72-eb34-4258-89f9-ba781921286a>